In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import nltk
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
nltk.download('punkt')
nltk.download('stopwords')
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import precision_score, recall_score, precision_recall_curve
from matplotlib import pyplot as plt
from sklearn.metrics import plot_precision_recall_curve
import numpy as np
from sklearn.model_selection import GridSearchCV

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ACER\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ACER\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# df = pd.read_csv("testdata.manual.2009.06.14.csv", sep=",")
df = pd.read_excel('text_rating_final.xlsx')

In [3]:
df.shape

(32436, 3)

In [4]:
df.head()

,"Не рациональная системность, а интуитивный позитивизм, хотя как-то коррелирующий с недедуцируемыми родовыми множествами неканторовской теории множеств Пола Коэна/ Мы сразу же сталкиваемся с двумя проблемами. Во-первых, если условиями философии служат истинностные процедуры, то это означает, что сама по себе она истин не производит. По сути дела, эта ситуация хорошо известна кто сможет процитировать хотя бы одно философское высказывание, о котором имело бы смысл говорить, что оно “истинно”? Но тогда что же именно в философии на кону? Во-вторых, мы предполагаем, что философия “одна”,—в том плане, что законно говорить о Философии, определять какой-либо текст как философский. В каком отношении находится это предполагаемое единство с множественностью условий? Каков этот узел четверицы (родовых процедур—матемы, поэмы, политического изобретения и любви) и одного (философии)? Я собираюсь показать, что на обе проблемы имеется один и тот же ответ, и содержится он в определении философии, представленный здесь как недейственная истинность при условии действенности истинного. Истинностные процедуры, или процедуры родовые, отличаются от накопления знаний своим событийным происхождением. Пока случается только то, что соответствует правилам некоего положения вещей, возможно, конечно же, познание, правильные высказывания, накопление знаний невозможна здесь истина. Истина парадоксальна тем, что она одновременно и внове, следовательно, — нечто редкостное, исключительное, и, затрагивая само бытие того, истиной чего является, наиболее прочна, наибо/17/лее, говоря онтологически, близка к исходному положению вещей.",0,Unnamed: 2
0,"Когда возникнут трудности, они тебе не помогут...",0,NaN
1,Кривая национализация это политический компром...,-1,NaN
2,Такой вид биологического оружия не действует н...,-2,NaN
3,В Эль-Кусейре /к западу от Хомса/ сирийские по...,0,NaN
4,"В том числе, ограничениями на продажу спиртных...",0,NaN


In [5]:
df.columns=[5, "a", 2]

In [6]:
df.head(5)

,5,a,2
0,"Когда возникнут трудности, они тебе не помогут...",0,NaN
1,Кривая национализация это политический компром...,-1,NaN
2,Такой вид биологического оружия не действует н...,-2,NaN
3,В Эль-Кусейре /к западу от Хомса/ сирийские по...,0,NaN
4,"В том числе, ограничениями на продажу спиртных...",0,NaN


In [7]:
df.shape

(32436, 3)

In [8]:
df.describe

<bound method NDFrame.describe of                                                        5   a    2
0      Когда возникнут трудности, они тебе не помогут...   0  NaN
1      Кривая национализация это политический компром...  -1  NaN
2      Такой вид биологического оружия не действует н...  -2  NaN
3      В Эль-Кусейре /к западу от Хомса/ сирийские по...   0  NaN
4      В том числе, ограничениями на продажу спиртных...   0  NaN
...                                                  ...  ..  ...
32431  Это помогло Соединенным Штатам прорубить окно ...   0  NaN
32432  Она уже вернулась на УИК. Член ПСГ Строгин от ...  -2  NaN
32433  Всего было две линии обороны: в первую входили...  -1  NaN
32434  5. Западный образ жизни несовместим с выживани...  -2  NaN
32435  Оригинал взят у в Светлая память - защитнику О...   0  NaN

[32436 rows x 3 columns]>

In [9]:
df_new = df.drop([2], axis=1)

In [10]:
df_new

,5,a
0,"Когда возникнут трудности, они тебе не помогут...",0
1,Кривая национализация это политический компром...,-1
2,Такой вид биологического оружия не действует н...,-2
3,В Эль-Кусейре /к западу от Хомса/ сирийские по...,0
4,"В том числе, ограничениями на продажу спиртных...",0
...,...,...
32431,Это помогло Соединенным Штатам прорубить окно ...,0
32432,Она уже вернулась на УИК. Член ПСГ Строгин от ...,-2
32433,Всего было две линии обороны: в первую входили...,-1
32434,5. Западный образ жизни несовместим с выживани...,-2


In [11]:
df_new.loc[(df_new.a == - 2), "a"] = -1

In [12]:
df_new.loc[(df_new.a == 2), "a"] = 1

In [13]:
df_new["a"].value_counts()

0               16967
-1              12264
1                3188
 совершенный        4
Name: a, dtype: int64

In [14]:
for j in range(1000, 32436):
    df_new = df_new.drop(index=[j])

In [15]:
df_new

,5,a
0,"Когда возникнут трудности, они тебе не помогут...",0
1,Кривая национализация это политический компром...,-1
2,Такой вид биологического оружия не действует н...,-1
3,В Эль-Кусейре /к западу от Хомса/ сирийские по...,0
4,"В том числе, ограничениями на продажу спиртных...",0
...,...,...
995,В этот момент вы соединили стадию зарождения м...,0
996,В тантрах этого раздела упор делается скорее н...,-1
997,Я потрясен и возмущен в связи с усиливающимися...,0
998,Члены египетских и иорданских такфиритских гру...,0


In [16]:
df_new.describe()

,5,a
count,998,1000
unique,827,3
top,"бактериофобия, верминофобия, вермифобия, гельм...",0
freq,7,518


In [17]:
for c in df_new[df_new["a"] == 1][5].head(5):
    print(c)

Великий писатель беспрестанно доказывал несостоятельность утопий разума, несоответствие любых рассудочных построений таинственной и беспредельной сущности человека. Достоевский очень глубоко выразил кризис модерна ещё до того, как он перешел в острую фазу. Прошедший через страсть по модерну (западничество сороковых), Достоевский со свойственной ему гениальностью понял, что модерный мир ведет человека к краю бездны. После смерти писателя его слава и влияние постоянно росли — углублялся и обострялся кризис модерна, выразителем которого был Достоевский. Разум бунтует против веры, личность бунтует против разума, смещаются представления о добре и зле, дьявол с богом борется — мир разорван, человек разорван. В Европе роль провозвестника кризиса взял на себя Ф. Ницше, который в рефлексивной форме выразил то кризисное мироощущение, которое художественно отразил Достоевский.
С одной стороны боевики захватили большую часть его кварталов, с другой - перестрелки на восточных окраинах еще продолжал

In [18]:
for c in df_new[df_new["a"] == 0][5].head(5):
    print(c)

Когда возникнут трудности, они тебе не помогут. Они не более, чем скатерть приятных концепций, ты еще не развязал узел концептуального мышления. Это подобно тому, как если бы в тебе была латентная болезнь. Сейчас ты в блаженстве, но тебе это не поможет. Поскольку ты не проник в самое ядро, зомби заблуждения до сих пор бродит вокруг. 1 Если ты посчитаешь медитационное переживание высшим, то ты не сможешь достичь воззрения, так как погряз в концепциях. Если ты очаруешься крупицей самадхи, думая что нет ничего выше этого и что вот это и есть совершенное самадхи, ты не сможешь прорваться сквозь деятельность концептуального мышления.
В Эль-Кусейре /к западу от Хомса/ сирийские пограничники преградили путь нескольким бандам террористов-наемников, пытавшимся прорваться в горный район Калямун. Сирийская армия перекрывает все подступы к этому крупному плацдарму боевиков к северо-западу от Дамаска. Под Алеппо сирийские военнослужащие освободили город Телль-Хасиль, населенный курдами. Информагент

In [19]:
for c in df_new[df_new["a"] == -1][5].head(5):
    print(c)

Кривая национализация это политический компромисс. С одной стороны, многочисленные рантье, возникшие вследствие приватизации, сохранили возможность и дальше получать свою ренту. Этот паразитический класс даже существенно увеличился в размере, пополнившись многочисленными представителями силовой бюрократии, не успевшими к первой раздаче. С другой стороны, все они превратились в условных держателей активов, распоряжающихся ими с разрешения правительства, которое накладывает на них разнообразные обременения как социального, так и коррупционного характера.Эта уродливая система, основанная на слегка задрапированном голом насилии, не решая ни одной из проблем, порожденных приватизацией, добавила к ним новые проблемы, ставшие следствием порождаемого ею правового беспредела. Именно попытка осуществить скрытую национализацию привела к окончательному превращению России в мафиозное государство. Путин лечил Россию, но не вылечил.
Такой вид биологического оружия не действует наверняка, как, к приме

In [20]:
train_df, test_df = train_test_split(df_new, test_size=200)

In [21]:
sentence_example = df_new.iloc[1][5]
tokens = word_tokenize(sentence_example, language="russian")
tokens_without_punctuation = [i for i in tokens if i not in string.punctuation]
russian_stop_words = stopwords.words("russian")
tokens_without_stop_words_and_punctuation = [i for i in tokens_without_punctuation if i not in russian_stop_words]
snowball = SnowballStemmer(language="russian")
stemmed_tokens = [snowball.stem(i) for i in tokens_without_stop_words_and_punctuation]

In [22]:
print(f"Исходный текст: {sentence_example}")
print("-----------------")
print(f"Токены: {tokens}")
print("-----------------")
print(f"Токены без пунктуации: {tokens_without_punctuation}")
print("-----------------")
print(f"Токены без пунктуации и стоп слов: {tokens_without_stop_words_and_punctuation}")
print("-----------------")
print(f"Токены после стемминга: {stemmed_tokens}")
print("-----------------")

Исходный текст: Кривая национализация это политический компромисс. С одной стороны, многочисленные рантье, возникшие вследствие приватизации, сохранили возможность и дальше получать свою ренту. Этот паразитический класс даже существенно увеличился в размере, пополнившись многочисленными представителями силовой бюрократии, не успевшими к первой раздаче. С другой стороны, все они превратились в условных держателей активов, распоряжающихся ими с разрешения правительства, которое накладывает на них разнообразные обременения как социального, так и коррупционного характера.Эта уродливая система, основанная на слегка задрапированном голом насилии, не решая ни одной из проблем, порожденных приватизацией, добавила к ним новые проблемы, ставшие следствием порождаемого ею правового беспредела. Именно попытка осуществить скрытую национализацию привела к окончательному превращению России в мафиозное государство. Путин лечил Россию, но не вылечил.
-----------------
Токены: ['Кривая', 'национализация

In [23]:
snowball = SnowballStemmer(language="russian")
russian_stop_words = stopwords.words("russian")

def tokenize_sentence(sentence: str, remove_stop_words: bool = True):
    tokens = word_tokenize(sentence, language="russian")
    tokens = [i for i in tokens if i not in string.punctuation]
    if remove_stop_words:
        tokens = [i for i in tokens if i not in russian_stop_words]
    tokens = [snowball.stem(i) for i in tokens]
    return tokens

In [24]:
tokenize_sentence(sentence_example)

['крив',
 'национализац',
 'эт',
 'политическ',
 'компромисс',
 'с',
 'одн',
 'сторон',
 'многочислен',
 'рант',
 'возникш',
 'вследств',
 'приватизац',
 'сохран',
 'возможн',
 'дальш',
 'получа',
 'рент',
 'этот',
 'паразитическ',
 'класс',
 'существен',
 'увелич',
 'размер',
 'пополн',
 'многочислен',
 'представител',
 'силов',
 'бюрократ',
 'успевш',
 'перв',
 'раздач',
 'с',
 'сторон',
 'преврат',
 'условн',
 'держател',
 'актив',
 'распоряжа',
 'им',
 'разрешен',
 'правительств',
 'котор',
 'накладыва',
 'разнообразн',
 'обременен',
 'социальн',
 'коррупцион',
 'характера.эт',
 'уродлив',
 'систем',
 'основа',
 'слегк',
 'задрапирова',
 'гол',
 'насил',
 'реш',
 'одн',
 'пробл',
 'порожден',
 'приватизац',
 'добав',
 'нов',
 'проблем',
 'ставш',
 'следств',
 'порожда',
 'е',
 'правов',
 'беспредел',
 'имен',
 'попытк',
 'осуществ',
 'скрыт',
 'национализац',
 'привел',
 'окончательн',
 'превращен',
 'росс',
 'мафиозн',
 'государств',
 'путин',
 'леч',
 'росс',
 'вылеч']

In [25]:
vectorizer = TfidfVectorizer(tokenizer=lambda x: tokenize_sentence(x, remove_stop_words=True))

In [26]:
features = vectorizer.fit_transform(train_df[5].values.astype('U'))

In [29]:
model = LogisticRegression(random_state=0)
model.fit(features, train_df["a"])

ValueError: Unknown label type: 'unknown'

In [ ]:
model.predict(features[0])

array([0], dtype=int64)

In [ ]:
train_df[5].iloc[0]

'NOOOOOOO my DVR just died and I was only half way through the EA presser. Hate you Time Warner'

In [ ]:
model_pipeline = Pipeline([
    ("vectorizer", TfidfVectorizer(tokenizer=lambda x: tokenize_sentence(x, remove_stop_words=True))),
    ("model", LogisticRegression(random_state=0))
]
)

In [ ]:
model_pipeline.fit(train_df[5], train_df[0])

Pipeline(steps=[('vectorizer',
                 TfidfVectorizer(tokenizer=<function <lambda> at 0x000001EA8899DF70>)),
                ('model', LogisticRegression(random_state=0))])

In [ ]:
model_pipeline.predict(["Hello, are you okay?"])

array([4], dtype=int64)

In [ ]:
model_pipeline.predict(["Hello"])

array([4], dtype=int64)

In [ ]:
model_pipeline.predict(["fuck you"])

array([0], dtype=int64)

In [ ]:
model_pipeline.predict(["hello, I really don't like this"])

array([0], dtype=int64)